In [61]:
library(rhdf5)
library(Matrix)
library(SingleCellExperiment)

In [62]:
mydir = "/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/in_vivo_comparison/"

In [63]:
dir_scanpy = "/nfs/leia/research/stegle/dseaton/hipsci/singlecell_neuroseq/data/data_processed/pool1_17_subsample_v1/"

In [64]:
filename = paste0(dir_scanpy,"pool1_17_subsample_v1.scanpy.w_metadata.w_celltype.scanpy.h5")

In [65]:
cbind(h5ls(filename)$group,
h5ls(filename)$name)

/,X
/X,data
/X,indices
/X,indptr
/,obs
/,obsm
/obsm,X_umap
/,uns
/uns,batch_categories
/uns,celltype_categories
/uns,donor_id_categories


In [66]:
# unique(h5read(file = filename, "obs")["celltype"])

In [67]:
# h5read(file = filename, "uns")$celltype_categories

In [68]:
ind = h5read(file = filename, "obs")$index # cell names
var = h5read(file = filename, "var") # gene names

In [69]:
head(ind)

[1] "TTCCCAGGTTCCATGA-1-0" "AACCGCGAGGTCATCT-1-0" "TCACAAGTCTTATCTG-1-0"
[4] "TACCTTACACGGTTTA-1-0" "CCTAGCTAGCGCCTTG-1-0" "CAAGATCAGGCTCTTA-1-0"

In [70]:
var$index[grep("NANOG",var$index)]

[1] "NANOGNB" "NANOG"

In [71]:
X = h5read(file = filename, "X")
X_data = as.integer(X$data)
X_indptr = as.integer(X$indptr)
X_indices = as.integer(X$indices)
rm(X)

In [72]:
sMat = sparseMatrix(
i = X_indices,
p = X_indptr,
x = X_data,
dims = c(length(var[[1]]), length(ind)),
index1 = FALSE
)
rownames(sMat) <- as.character(var[[1]])
colnames(sMat) <- as.character(ind)

In [73]:
rownames(sMat)[grep("SOX",rownames(sMat))]

[1] "QSOX1"     "SOX13"     "SOX11"     "SOX14"     "SOX2"      "SOX30"    
 [7] "SOX4"      "SOX3"      "SOX7"      "SOX7-1"    "SOX17"     "QSOX2"    
[13] "SOX6"      "SOX5"      "SOX21"     "SOX21-AS1" "SOX1"      "SOX8"     
[19] "SOX15"     "SOX9"      "SOX12"     "SOX18"     "SOX10"

In [74]:
R_filename = paste0(mydir,"neuroseq_all_20pct.Rds")
saveRDS(sMat, file = R_filename)

In [ ]:
cData_raw = h5read(file = filename, "obs")
# head(cData_raw)

In [ ]:
# now the column data information
colinfo_all = apply(cbind(h5ls(filename)$group,
    h5ls(filename)$name)
    ,1,paste0, collapse = "/")
colinfo_all

In [ ]:
colinfo <- grep("categories|//",grep("obs",colinfo_all, value = TRUE), invert = TRUE, value = TRUE)
colinfo

In [ ]:
for (col in colinfo){
    cat = gsub("/uns/","",gsub("_categories","",col))
    cats = h5read(file = filename, col)
    print(cats)
    #print(head(cData_raw[[cat]]))
    cData_raw[[cat]] = cats[cData_raw[[cat]]+1]
}

In [ ]:
head(cData_raw)

In [29]:
cData_raw$umap1 = h5read(file = filename, "obsm")$X_umap[1,]
cData_raw$umap2 = h5read(file = filename, "obsm")$X_umap[2,]

In [30]:
rownames(cData_raw) = cData_raw$index

In [31]:
head(cData_raw)

,index,batch,donor_id,log1p_n_genes_by_counts,log1p_total_counts,log1p_total_counts_mito,n_genes_by_counts,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,...,total_counts,total_counts_mito,cluster_id,time_point,pool_id,treatment,n_counts_all,celltype,umap1,umap2
TTCCCAGGTTCCATGA-1-0,TTCCCAGGTTCCATGA-1-0,0,HPSI0214i-eiwy_1,7.893572,9.321613,4.077538,2679,53.54331,63.84216,74.86578,...,11176,58,3,D11,pool1,NONE,11176,FPP,-7.510658,6.5592762
AACCGCGAGGTCATCT-1-0,AACCGCGAGGTCATCT-1-0,0,HPSI0114i-eipl_1,8.214194,9.801510,5.147494,3692,51.97121,61.66667,72.53045,...,18060,171,5,D11,pool1,NONE,18060,P_FPP,-10.086959,3.0156652
TCACAAGTCTTATCTG-1-0,TCACAAGTCTTATCTG-1-0,0,HPSI1113i-podx_1,8.319474,10.024244,5.318120,4102,51.51112,61.17611,71.90020,...,22566,203,5,D11,pool1,NONE,22566,P_FPP,-9.206473,0.9118701
TACCTTACACGGTTTA-1-0,TACCTTACACGGTTTA-1-0,0,HPSI0614i-liqa_1,7.825245,9.297252,4.744932,2502,58.59540,67.71798,77.77574,...,10907,114,3,D11,pool1,NONE,10907,FPP,-8.961995,8.4147848
CCTAGCTAGCGCCTTG-1-0,CCTAGCTAGCGCCTTG-1-0,0,HPSI0114i-eipl_1,8.029433,9.451638,4.718499,3069,50.48712,60.40226,71.76304,...,12728,111,5,D11,pool1,NONE,12728,P_FPP,-9.453191,3.0499935
CAAGATCAGGCTCTTA-1-0,CAAGATCAGGCTCTTA-1-0,0,HPSI1113i-podx_1,8.204125,9.751094,5.043425,3655,50.46005,59.99884,71.23224,...,17172,154,6,D11,pool1,NONE,17172,P_FPP,-9.914284,-2.1984845


In [32]:
cData <- as.data.frame(cData_raw)
cData_R_filename = paste0(mydir,"neuroseq_all_25pct_cData.Rds")
saveRDS(cData, file = cData_R_filename)

In [33]:
# build the sce object
sce <- SingleCellExperiment(
assays = list(counts = sMat),
colData = cData)
sce

class: SingleCellExperiment 
dim: 32738 205416 
metadata(0):
assays(1): counts
rownames(32738): MIR1302-10 FAM138A ... AC002321.2 AC002321.1
rowData names(0):
colnames(205416): TTCCCAGGTTCCATGA-1-0 AACCGCGAGGTCATCT-1-0 ...
  GCGCAGTAGGAATGGA-1-150 GTCCTCATCTAACTTC-1-150
colData names(23): index batch ... umap1 umap2
reducedDimNames(0):
spikeNames(0):
altExpNames(0):

In [36]:
sce_R_filename = paste0(mydir,"neuroseq_all_25pct_sce.Rds")
saveRDS(sce, file = sce_R_filename)

In [35]:
rownames(sce)[grep("SOX",rownames(sce))]

[1] "QSOX1"     "SOX13"     "SOX11"     "SOX14"     "SOX2"      "SOX30"    
 [7] "SOX4"      "SOX3"      "SOX7"      "SOX7-1"    "SOX17"     "QSOX2"    
[13] "SOX6"      "SOX5"      "SOX21"     "SOX21-AS1" "SOX1"      "SOX8"     
[19] "SOX15"     "SOX9"      "SOX12"     "SOX18"     "SOX10"